In [2]:
import json as json

with open("recipes_data.json", "r") as read_file:
    data = json.load(read_file)

In [3]:
j = len(data)
str(j)

'994'

In [10]:
import pprint as pp
import requests

host = 'api.novasearch.org'
port = 443
user = 'user201' # Add your user name here.
password = 'Lrr1531' # Add your user password here. For testing only. Don't store credentials in code. 
index_name = user 


In [5]:
data['0']

{'displayName': 'How To Make Chicken Parmesan',
 'description': 'Master the classic dish of chicken Parmesan by starting with the chicken, choosing a marinara sauce you love, and using a trio of cheese. ',
 'canonicalName': None,
 'prepTimeMinutes': None,
 'cookTimeMinutes': None,
 'totalTimeMinutes': None,
 'cookingMethod': None,
 'difficultyLevel': None,
 'images': [{'url': 'https://m.media-amazon.com/images/S/alexa-kitchen-msa-na-prod/recipes/thekitchn/016aa4923f044e1bad4cb2802f04133f7cf787b9bbf4fceb52438ecb70b28d89.jpg',
   'hdURL': None,
   'fourKURL': None,
   'description': None,
   'type': 'UNKNOWN'}],
 'videos': [],
 'tools': [{'displayName': 'Meat mallet or small saucepan', 'images': []},
  {'displayName': 'Paper towels', 'images': []},
  {'displayName': 'shallow bowls or 8-inch square dishes', 'images': []},
  {'displayName': 'Large plate', 'images': []},
  {'displayName': 'Plastic wrap or wax paper', 'images': []},
  {'displayName': 'Tongs', 'images': []},
  {'displayName':

In [43]:

# Title and description
data['0']['displayName']
data['0']['description']

# Tools
data['0']['tools'][0]
data['0']['tools'][1]
data['0']['tools'][2]

# Ingredients
data['0']['ingredients'][0]
data['0']['ingredients'][1]
data['0']['ingredients'][2]

# Steps
data['0']['instructions'][0]
data['0']['instructions'][1]
data['0']['instructions'][2]


TypeError: list indices must be integers or slices, not str

### Recipes Index


In [11]:
import pprint as pp
from opensearchpy import OpenSearch
from opensearchpy import helpers

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = (user, password),
    url_prefix = 'opensearch',
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False
)

if client.indices.exists(index_name):

    resp = client.indices.open(index = index_name)
    print(resp)

    print('\n----------------------------------------------------------------------------------- INDEX SETTINGS')
    settings = client.indices.get_settings(index = index_name)
    pp.pprint(settings)

    print('\n----------------------------------------------------------------------------------- INDEX MAPPINGS')
    mappings = client.indices.get_mapping(index = index_name)
    pp.pprint(mappings)

    print('\n----------------------------------------------------------------------------------- INDEX #DOCs')
    print(client.count(index = index_name))
    

{'acknowledged': True, 'shards_acknowledged': True}

----------------------------------------------------------------------------------- INDEX SETTINGS
{'user201': {'settings': {'index': {'creation_date': '1711539152656',
                                    'number_of_replicas': '1',
                                    'number_of_shards': '1',
                                    'provided_name': 'user201',
                                    'uuid': 'WuAh8_gpRwGPylc02OOOtw',
                                    'version': {'created': '135238227'}}}}}

----------------------------------------------------------------------------------- INDEX MAPPINGS
{'user201': {'mappings': {'properties': {'contents': {'fields': {'keyword': {'ignore_above': 256,
                                                                             'type': 'keyword'}},
                                                      'type': 'text'},
                                         'doc_id': {'fields': {'keyword': {'i

In [51]:
index_body = {
    "mappings": {
        "properties": {
            "recipe_title": {
                "type": "text"
            },
           
            "description": {
                "type": "text"
            },
            "ingredients": {
                "type": "keyword"
            },
            "time": {
                "type": "integer"
            },
            "recipe": {
                "type": "object"
            },
            
        }
    }
}

if client.indices.exists(index=index_name):
    print("Index already existed. Nothing to be done.")
else:        
    response = client.indices.create(index_name, body=index_body)
    print('\nCreating index:')
    print(response)


Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'user201'}


In [53]:
docs = data['0']
doc = {
    "recipe_title": docs['displayName'],
    #"title_embedding": docs['title_embedding'],
    "description": docs['description'],
    "ingredients": docs['ingredients'],
    "time": docs['totalTimeMinutes'],
    "recipe": docs
}
resp = client.index(index=index_name, id=1, body=doc)

print(resp['result'])
print(docs[0])


RequestError: RequestError(400, 'mapper_parsing_exception', "failed to parse field [ingredients] of type [keyword] in document with id '7OnZf44B7xTIhwxmAhxd'. Preview of field's value: '{displayText=½ teaspoon of red pepper flakes (optional), ingredientId=nx_item_00780, componentIndex=0, unit=TEASPOON, images=[], ingredient=red pepper flakes, quantity=0.5, staple=false, asinOverride=null, brand=null, productOverride=null, preparation=null}'")